In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [1]:
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=Steals&agg=PerGame"
years = list(range(2001, 2022))

In [4]:
for year in years:
    with open("steals_leaders_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)

In [5]:
dfs = []
for year in years:
    with open("steals_leaders_htmls/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    steals_data = pd.read_html(str(player_table))[0]
    steals_data["Year"] = year
    dfs.append(steals_data)

In [6]:
steals_data = pd.concat(dfs)
steals_data

,Rank,Player,Team,GamesG,StealsST,Ave.,Per 40 min.40m,Year
0,1,"GINOBILI, EMANUELGINOBILI, E.",VIRKinder Virtus BolognaKinder Virtus Bologna,22,58,2.64,3.70,2001
1,2,"HUGGINS, ROGERHUGGINS, R.",CHASpirou BasketSpirou Basket,14,32,2.29,3.07,2001
2,3,"JARIC, MARKOJARIC, M.",VIRKinder Virtus BolognaKinder Virtus Bologna,21,47,2.24,3.17,2001
3,4,"PHELPS, DERRICKPHELPS, D.",BERAlba BerlinAlba Berlin,13,29,2.23,2.75,2001
4,5,"BIGGS, RALPHBIGGS, R.",OOSTelindus OostendeTelindus Oostende,14,30,2.14,2.33,2001
...,...,...,...,...,...,...,...,...
25,25,"GIEDRAITIS, ROKASGIEDRAITIS, R.",BKNBitci Baskonia Vitoria-GasteizBitci Baskoni...,29,29,1.00,1.39,2021
26,25,"BACON, DWAYNEBACON, D.",ASMAS MonacoAS Monaco,29,29,1.00,1.58,2021
27,25,"WALDEN, COREYWALDEN, C.",BAYFC Bayern MunichFC Bayern Munich,25,25,1.00,1.79,2021
28,25,"JEKIRI, TONYEJEKIRI, T.",UNKUNICS KazanUNICS Kazan,24,24,1.00,1.63,2021


In [7]:
engine = create_engine('sqlite:///steals_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
steals_data.to_sql(
    name = "steals_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

1050

In [9]:
steals_leaders_select = session.execute('select * from "steals_leaders"')
steals_leaders_select.all()

[(0, 1, 'RICH, JAMEILRICH, J.', 'LUGLugano SnakesLugano Snakes', 10, 37, 3.7, 4.42, 2001),
 (1, 1, 'MARIC, IVICAMARIC, I.', 'ZADZadarZadar', 10, 37, 3.7, 4.65, 2001),
 (2, 3, 'PITTIS, RICCARDOPITTIS, R.', 'TREBenetton BasketBenetton Basket', 14, 47, 3.36, 4.01, 2001),
 (3, 4, 'GINOBILI, EMANUELGINOBILI, E.', 'VIRKinder Virtus BolognaKinder Virtus Bologna', 22, 64, 2.91, 3.91, 2001),
 (4, 5, 'HAMILTON, DERECKHAMILTON, D.', 'PETSaint Petersburg LionsSaint Petersburg Lions', 10, 23, 2.3, 2.38, 2001),
 (5, 6, 'FUCKA, GREGORFUCKA, G.', 'FORPAF BolognaPAF Bologna', 17, 39, 2.29, 2.75, 2001),
 (6, 7, 'OBERTO, FABRICIOOBERTO, F.', 'BKNTau Ceramica Vitoria-GasteizTau Ceramica Vitoria-Gasteiz', 22, 44, 2.0, 2.85, 2001),
 (7, 7, 'TOMASEVIC, DEJANTOMASEVIC, D.', 'BUDBuducnostBuducnost', 12, 24, 2.0, 2.34, 2001),
 (8, 7, 'LEE, ARTHURLEE, A.', 'CIBCibona VIPCibona VIP', 12, 24, 2.0, 2.55, 2001),
 (9, 7, 'MATTHEWS, LEEMATTHEWS, L.', 'LUGLugano SnakesLugano Snakes', 10, 20, 2.0, 2.56, 2001),
 (10, 7, 